In [ ]:
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-PLAN OF ATTACK=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
Run Remmina as ADMINISTRATOR
  RDP to new Admin Station/box

Get new admin station setup
  Run Powershell as ADMINISTRATOR
  Run CMD as ADMINISTRATOR
  Get MobaXterm, install, run as ADMINISTRATOR
  Setup Linux Box
  Setup Windows Box
  Start with Windows Box
  
  # Install SYSINTERNALS
net use * http://live.sysinternals.com



Check ports
Check processes
get PID and PPIDs
Check registries
Check Runs/Autoruns/RunOnce/Startup Stuff
Check Filesystem

-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=BREAKDOWN-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=

# find ports running malicious activity, use the following PowerShell command

Get-NetTCPConnection | Where-Object {$_.State -eq "Established"} | Select-Object LocalAddress, LocalPort, RemoteAddress, RemotePort, State, OwningProcess | Sort-Object LocalPort
#  lists all established TCP connections and their owning processes. You can then look for any suspicious processes running on the ports.

# check the registry for malware, use the following PowerShell command
Get-ChildItem -Path HKLM:\Software\Microsoft\Windows\CurrentVersion\Run -ErrorAction SilentlyContinue | ForEach-Object {Get-ItemProperty $_.PSPath} | Where-Object {$_.PSChildName -ne $null}
# lists all the items in the "Run" key of the registry, which are executed at startup. You can look for any suspicious entries that may be related to malware

# check the task scheduler for malware, you can use the following PowerShell command
Get-ScheduledTask | Where-Object {$_.State -ne "Disabled"} | ForEach-Object {Get-ScheduledTaskInfo $_.TaskPath} | Where-Object {$_.Triggers.Count -gt 0}
#  lists all scheduled tasks that are enabled and have triggers. You can look for any suspicious tasks that may be related to malware

#check services for malware, you can use the following PowerShell command
Get-WmiObject Win32_Service | Where-Object {$_.StartMode -ne "Disabled" -and $_.State -ne "Stopped"} | Select-Object Name, DisplayName, StartMode, State, PathName, ProcessId
# lists all running services and their process IDs. You can look for any suspicious services that may be related to malware.


-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=     Windows    -=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=


-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-PORTS-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=

###### Looking for malicious activity on a port with NETSTAT ######

# Launch PowerShell or Command Prompt as an administrator.

netstat -ano

# to view all active network connections

# This will shows a list of all active connections, along with the process ID (PID) associated with each connection.

# Look for any connections that are associated with suspicious IP addresses or ports. Some common ports used by malware include 12345, 6666, 6667, 6668, and 6669 (IRC), 1433 (Microsoft SQL Server), 3389 (Remote Desktop Protocol), and 4444 (reverse shell).

# Once you have identified a suspicious connection, note the associated PID and use the following command to view more information about the process:

tasklist /fi "pid eq <PID>"

# Replace <PID> with the actual process ID you noted earlier.

# This will display the name of the process, the username under which it is running, and other details.

# If you suspect that the process is malicious, you can terminate it using the following command:

taskkill /pid <PID> /f

# Replace <PID> with the actual process ID you noted earlier

##### # Looking for malicious activity on a port with TCPVIEW # #####

'''Launch TCPView as an administrator.  (run from the drive or use the address *example * z:./TCPview)

Click the column header for the "Local Port" column to sort the list of connections by local port number.
Locate the connection you are interested in and note its process ID (PID) in the "PID" column.
Click the "Processes" menu and select "Show Process Information" (or press Ctrl+P).
In the "Process Properties" dialog box, locate the process with the matching PID and note its name and parent process ID (PPID).

Look for any connections that are associated with suspicious IP addresses or ports. You can sort the connections by clicking on the column headers, such as "Local Address" or "Remote Address".

Once you have identified a suspicious connection, double-click on it to view more information about the process.

If you suspect that the process is malicious, you can terminate it by right-clicking on the connection and selecting "End Process".
'''
# CimInstance lets you see the parent process ID
Get-CimInstance Win32_Process | select name,processid,ParentProcessId | sort processid


# use this to get process ID and Description
Get-CimInstance Win32_process | select name,processid,description | ft-wrap

-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=REGISTRY-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
# Searching for malware/maliciousness in Registry #
'''
search for suspicious keys in the following locations:

HKEY_LOCAL_MACHINE\Software\Microsoft\Windows\CurrentVersion\Run
HKEY_LOCAL_MACHINE\Software\Microsoft\Windows\CurrentVersion\RunOnce
HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Run
HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\RunOnce
'''
# Searching through both run and runOnce piping them to get the property values for each
Get-ChildItem -Path HKLM:\SOFTWARE\Microsoft\Windows\CurrentVersion\Run* | ForEach-Object {Get-ItemProperty $_.PSPath}

# search through the Windows Task Scheduler
Get-ScheduledTask | Get-ScheduledTaskInfo | Select-Object TaskName,LastRunTime,NextRunTime,LastTaskResult

# search through running services
Get-Service | Select-Object Name,DisplayName,Status,StartType | Where-Object {$_.StartType -ne "Disabled" -and $_.Status -eq "Running"}

# search through installed software
Get-ItemProperty HKLM:\Software\Wow6432Node\Microsoft\Windows\CurrentVersion\Uninstall\* | Select-Object DisplayName, DisplayVersion, Publisher, InstallDate

-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=AUTO STARTUPS=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
# Searching for processes that automatically start up # 
Get-ScheduledTask

# Using Autoruns #
'''
1. Run Autoruns as Admin or call it from cmd/pshell (.\Autoruns.exe)
2. filter the results by clicking on the "Options" menu and selecting "Hide Microsoft Entries" to reduce the clutter
3. Look for any entries that seem suspicious, such as ones with strange or unfamiliar names, locations, or descriptions. 
Suspicious entries may also have no publisher information or a publisher name that does not match the legitimate publisher of the software
4. Right-click on the suspicious entry and select "Jump to" to view the location of the file or registry key associated with the entry.
If the file is located in a non-standard location, such as a temporary folder, outside of System32, or the root of the C:\ drive, it may be an indication of malicious activity.
5. Check the entry's properties by double-clicking on it. Look for any command-line parameters, environment variables, or other details that may provide clues about the purpose of the entry.
Some malware may use obfuscated or encrypted command-line parameters to evade detection, so be sure to scrutinize the details carefully.
'''

'''
and watch for triggering mechanisms....
See the scheduling tasks 

Look at the NAME, and the path they're being run from... if it's not from the System32 directory... it's suspicious
'''

-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=Check Filesystem=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
# Use the Get-Process cmdlet to list all running processes and their associated file paths
Get-Process | Select-Object Name, Path | Sort-Object Path
#lists all running processes and their associated file paths, sorted by file path. You can look for any suspicious processes or file paths that may be related to malware.


##### # Using Autoruns # #####
'''
1. Launch Autoruns as Administrator
2. Click on the "Everything" tab to see all configured startup items.
3. Look for any suspicious file paths and investigate them further

Autoruns can also show you additional information about each startup item, 
such as its publisher and digital signature, which can help you determine if it is legitimate or not
'''


-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=EVENT LOGS=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=

PowerShell to investigate the Windows event logs::

PowerShell:
'''
Launch PowerShell as an administrator.
To view the Security log, type the following command and press Enter: Get-EventLog -LogName Security
To view the System log, type the following command and press Enter: Get-EventLog -LogName System
To view the Application log, type the following command and press Enter: Get-EventLog -LogName Application
You can filter events by using the -After and -Before parameters followed by a date and time in the format MM/DD/YYYY HH:MM:SS.
For example, to view events that occurred between 10:00 AM and 12:00 PM on January 1st, 2023, you would use the following command: Get-EventLog -LogName Security -After "01/01/2023 10:00:00" -Before "01/01/2023 12:00:00"
'''

-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=      Linux -SysV    -=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=

-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=BREAKDOWN-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=

file /sbin/init                # USE THIS ONE FIRST                 # find out whether you're running init or systemd (sysV or SysD)



#find ports running malicious activity
sudo netstat -tuln
#show you a list of all open ports on your system along with the process that is listening on each port

#check for malicious processes running on the ports
sudo lsof -i
#show you a list of all open network connections on your system along with the process that has opened each connection

#check the scheduled tasks for malware,
sudo crontab -l
#list of all scheduled tasks on your system

#check services for malware and malicious activity
sudo systemctl list-units --type=service
#list of all running services on your system

# Show everything in a directory
ls -shilat

-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-PORTS-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
#find ports running malicious activity
sudo netstat -tuln
#show you a list of all open ports on your system along with the process that is listening on each port

##### # Using TCPDump # #####
sudo tcpdump -i any -v
# capture all network traffic on all interfaces and display the captured packets in verbose mode


##### # Correlate Malicious Port Activity to a Service # #####
sudo lsof -i :<port number>                                       #Replace <port number> with the actual port number you want to investigate
# lists all the processes that are using the specified port, along with their associated service names and process IDs (PIDs)

-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=Services/Processes-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
# real-time information about running processes view of processes, PID, PPID, commands, etc...
htop

# Shows the list of processes in a tree -type format.. good to see the tree breakdown of processes from parent to child
ps -elf --forest

#check services for malware and malicious activity
systemctl list-units --type=service
#list of all running services on your system

#list all running processes on your system
ps aux
# list of all running processes on your system, along with their process IDs (PIDs)


#Once you have PIDs and PPIDs, you can start finding where they're coming from

sudo lsof -Pn | grep <PID>                                      # Replace <PID> with the process ID of the process you want to investigate
# show a list of all files and network connections that are being used by the specified process, along with their associated file descriptors

# Find the file paths associated with a process
sudo lsof -p <PID> | grep -E "^COMMAND|\<REG|\/"

-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=Profiles=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
'''
Navigate to the various users' directories cd ~[username]
Check the following for any suspicious or unfamiliar stuff
cat .bashrc
cat .profile
cat .bash_history
cat .ssh/authorized_keys


cat the below profiles to look for malware
'''
# Directories of importance
  /    # Root
  /bin   #
  /etc/environment      #can hold malicous scripts
  /etc/profile          # 
  /etc/bash.bashrc      # Profile to look at
  /etc/profile.d
  
  
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=SCHEDULED TASKS=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
# location for user crontab
/var/spool/cron/crontabs/

#view the cron jobs for all users on the system
ls -l /var/spool/cron/crontabs/



#list all scheduled tasks for the current user
crontab -l



System cron jobs           /etc/crontab

sudo ls -shilat /var/spook/cron/crontabs




*     *     *     *     *     command to be executed
-     -     -     -     -
|     |     |     |     |
|     |     |     |     +----- day of the week (0 - 6) (Sunday = 0)
|     |     |     +------- month (1 - 12)
|     |     +--------- day of the month (1 - 31)
|     +----------- hour (0 - 23)
+------------- minute (0 - 59)


-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=EVENT LOGS=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
# display system logs, including messages from the kernel, services, and applications

#This log file contains general system messages, including startup messages, kernel messages, and other important system events.
cat /var/log/messages

#This log file contains authentication-related messages, such as login attempts, su attempts, and other authentication events.
cat /var/log/auth.log

#This log file contains messages from various system services, including the kernel, system daemons, and applications.
cat /var/log/syslog

#This log file contains kernel ring buffer messages that are generated during the boot process.
cat /var/log/dmesg



-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=


-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=      Linux -SysD    -=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=

-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=BREAKDOWN-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=

#find ports running malicious activity

netstat -tuln
OR
sudo ss -tulnp
#show you a list of all open ports on your system along with the process that is listening on each port

#check for malicious processes running on the ports
sudo lsof -i
#show you a list of all open network connections on your system along with the process that has opened each connection

#check the scheduled tasks for malware,
sudo crontab -l

or

sudo systemctl list-timers

#list of all scheduled tasks on your system

#check services for malware and malicious activity
sudo systemctl list-units --type=service
#list of all running services on your system

#check for malicious entries in the systemd journal
sudo journalctl -xe
# list of all log entries related to system events, including any suspicious or malicious activity.




-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=EVENT LOGS=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
# display system logs, including messages from the kernel, services, and applications
journalctl -b






